In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    remove_stop_words=False
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10762.04it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5651.96it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3494010.71it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 11246.28it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5792.83it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 907956.27it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8647.61it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10e749358>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x1373f5da0>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
padding_callback = mz.dataloader.callbacks.BasicPadding()
fixed_padding_callback = mz.dataloader.callbacks.BasicPadding(
    fixed_length_left=10,
    fixed_length_right=100
)

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=fixed_padding_callback
)

In [8]:
model = mz.models.DRMMTKS()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = -1
model.params['top_k'] = 10
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DRMMTKS(
  (embedding): Embedding(16675, 100)
  (attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (mlp): Sequential(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=5, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=5, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
)
Trainable params:  1667663


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.945]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5124 - normalized_discounted_cumulative_gain@5(0.0): 0.5702 - mean_average_precision(0.0): 0.5384



[Iter-204 Loss-0.732]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5763 - normalized_discounted_cumulative_gain@5(0.0): 0.6275 - mean_average_precision(0.0): 0.5909



[Iter-306 Loss-0.541]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6044 - normalized_discounted_cumulative_gain@5(0.0): 0.6624 - mean_average_precision(0.0): 0.6152



[Iter-408 Loss-0.429]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6377 - normalized_discounted_cumulative_gain@5(0.0): 0.6905 - mean_average_precision(0.0): 0.6487



[Iter-510 Loss-0.343]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6403 - normalized_discounted_cumulative_gain@5(0.0): 0.7003 - mean_average_precision(0.0): 0.6522



[Iter-612 Loss-0.282]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6626 - normalized_discounted_cumulative_gain@5(0.0): 0.7128 - mean_average_precision(0.0): 0.669



[Iter-714 Loss-0.237]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6654 - normalized_discounted_cumulative_gain@5(0.0): 0.7167 - mean_average_precision(0.0): 0.671



[Iter-816 Loss-0.174]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.658 - normalized_discounted_cumulative_gain@5(0.0): 0.7082 - mean_average_precision(0.0): 0.6618



[Iter-918 Loss-0.139]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6669 - normalized_discounted_cumulative_gain@5(0.0): 0.7173 - mean_average_precision(0.0): 0.6742



[Iter-1020 Loss-0.087]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6633 - normalized_discounted_cumulative_gain@5(0.0): 0.7102 - mean_average_precision(0.0): 0.6691

Cost time: 378.9530117511749s
